In [2]:
import sqlite3
import pandas as pd
from pathlib import Path

In [3]:
# Set options to display all columns and a larger number of rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50) # You can adjust the number of rows as needed
pd.set_option('display.width', 1000) # Optional: Increase the total display width

In [4]:
# Get the current working directory, which is 'src_data_loader'
notebook_dir = Path.cwd()
print(f"Notebook directory (current working directory): {notebook_dir}")

# Go up one level to the parent ('FLOODGPT'), then into the 'db' folder
db_path = notebook_dir.parent / "db" / "analytics.db"
print(f"Correct database path: {db_path}")

# This connection will now work correctly
conn = sqlite3.connect(db_path)
print("Connection successful!")



Notebook directory (current working directory): c:\DEV\FLOODGPT\src_data_loader
Correct database path: c:\DEV\FLOODGPT\db\analytics.db
Connection successful!


In [13]:
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(cpes_projects);")
for col in cursor.fetchall():
    print(col[1])

constructor_name
license_no
project
date_eval
implem_agency
amount
duration_cd
status
workmanship
materials
time
facility
environmental_safety_health
resource_deployment
cpes_visit_rating
final_rating
qualitative_desc
pdf_file
is_bad_row


In [5]:
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(flood_control_projects);")
for col in cursor.fetchall():
    print(col[1])

global_id
infra_year
region
province
municipality
implementing_office
project_id
project_description
project_component_id
project_component_description
program
typeof_work
infra_type
longitude
latitude
contract_id
abc
contract_cost
completion_date_original
completion_year
contractor
object_id
creation_date
creator
edit_date
editor
funding_year
legislative_district
district_engineering_office
abc_string
contract_cost_string
completion_date_actual
start_date
type
slug
geo


In [18]:
sql_query = """
SELECT distinct count ( constructor_name)
FROM cpes_projects
ORDER BY 1 asc
"""

df = pd.read_sql(sql_query, conn)
print(df)

   count ( constructor_name)
0                      13753


What are the top 5 flood control projects by budget? 

In [15]:
sql_query = """
SELECT
  project_description,
  abc
FROM flood_control_projects
ORDER BY
  abc DESC
LIMIT 5;
"""

df = pd.read_sql(sql_query, conn)
print(df)

                                 project_description          abc
0  Construction of River Bank Protection along Al...  310576313.2
1  Construction of River Bank Protection along Ma...  310576313.2
2  Construction of River Bank Protection along Ba...  310576313.2
3  Construction of River Bank Protection along Al...  310576313.2
4  Construction of Road Dike/Esplanade along Siti...  289500000.0


Which regions have the most completed projects?  

In [8]:
sql_query = """
SELECT
  region,
  COUNT(project_id) AS completed_projects_count
FROM flood_control_projects
WHERE
  completion_date_actual IS NOT NULL
GROUP BY
  region
ORDER BY
  completed_projects_count DESC;
"""

df = pd.read_sql(sql_query, conn)
print(df)

                              region  completed_projects_count
0                         Region III                      1617
1            National Capital Region                      1058
2                           Region V                       866
3                           Region I                       851
4                        Region IV-A                       829
5                          Region II                       628
6                         Region VII                       614
7                        Region VIII                       517
8                          Region VI                       505
9                        Region IV-B                       498
10                          Region X                       428
11                         Region XI                       356
12  Cordillera Administrative Region                       352
13                       Region XIII                       302
14                        Region XII                   

Which regions have the highest total budget allocation?

In [5]:
sql_query = """
SELECT
  region,
  SUM(abc) AS total_budget_allocation
FROM flood_control_projects
GROUP BY
  region
ORDER BY
  total_budget_allocation DESC;
"""

df = pd.read_sql(sql_query, conn)
print(df)

                              region  total_budget_allocation
0                         Region III             9.858121e+10
1                           Region V             5.640419e+10
2            National Capital Region             5.360433e+10
3                        Region IV-A             4.095329e+10
4                           Region I             3.992018e+10
5                        Region IV-B             3.651700e+10
6                          Region II             3.650277e+10
7                         Region VII             3.629472e+10
8                           Region X             2.782387e+10
9                          Region VI             2.529625e+10
10                       Region VIII             2.490216e+10
11                       Region XIII             2.410276e+10
12                         Region XI             2.131306e+10
13  Cordillera Administrative Region             1.791211e+10
14                        Region XII             1.713341e+10
15      

Top 5 flood control projects by ABC?

In [10]:
sql_query = """
SELECT
  project_description,
  abc
FROM flood_control_projects
ORDER BY
  abc DESC
LIMIT 5;
"""

df = pd.read_sql(sql_query, conn)
print(df)

                                 project_description          abc
0  Construction of River Bank Protection along Al...  310576313.2
1  Construction of River Bank Protection along Ma...  310576313.2
2  Construction of River Bank Protection along Ba...  310576313.2
3  Construction of River Bank Protection along Al...  310576313.2
4  Construction of Road Dike/Esplanade along Siti...  289500000.0


Average Contract Cost by Year

In [60]:
sql_query = """
SELECT infra_year,  printf('%,.2f',CAST(AVG(contract_cost) AS INT)) AS Avg_Contract_Cost
FROM flood_control_projects 
WHERE contract_cost IS NOT NULL AND contract_cost > 0
GROUP BY infra_year
ORDER BY 1 DESC;
"""

df = pd.read_sql(sql_query, conn)
print(df)

   infra_year Avg_Contract_Cost
0        2025     41,446,655.00
1        2024     61,449,634.00
2        2023     58,631,254.00
3        2022     49,267,565.00
4        2021     49,107,776.00
5        2020     69,695,056.00
6        2019    111,942,309.00
7        2018    104,965,936.00


Projects by Infrastructure Year

In [61]:
sql_query = """
SELECT 
    infra_year, 
    COUNT(DISTINCT global_id) AS Project_Count, 
    printf('%,.2f',SUM(contract_cost)) AS Total_Cost 
FROM flood_control_projects 
GROUP BY infra_year 
ORDER BY infra_year DESC;
"""

df = pd.read_sql(sql_query, conn)
print(df)

   infra_year  Project_Count          Total_Cost
0        2025             35    1,450,632,948.00
1        2024           2354  144,652,439,107.00
2        2023           3466  203,215,927,213.00
3        2022           3334  164,258,062,059.00
4        2021            620   30,446,821,662.00
5        2020             36    2,509,022,023.00
6        2019              3      335,826,928.00
7        2018              7      734,761,554.00


Completion Rate by Completion Year

In [62]:
sql_query = """
SELECT 
    completion_year, 
    COUNT(DISTINCT global_id) AS Completed_Projects 
FROM flood_control_projects 
WHERE completion_date_actual IS NOT NULL 
GROUP BY completion_year ORDER BY completion_year DESC;
"""

df = pd.read_sql(sql_query, conn)
print(df)


   completion_year  Completed_Projects
0             2025                 376
1             2024                3120
2             2023                3828
3             2022                2531


On-Time vs. Delayed Projects by Contractors

In [63]:
sql_query = """
SELECT 
    contractor,
    COUNT(project_id) as Total_Projects,
    SUM(CASE WHEN completion_date_actual <= completion_date_original THEN 1 ELSE 0 END) AS On_Time_Projects, 
    SUM(CASE WHEN completion_date_actual > completion_date_original THEN 1 ELSE 0 END) AS Delayed_Projects 
FROM flood_control_projects 
WHERE completion_date_actual IS NOT NULL
GROUP BY contractor
ORDER BY Delayed_Projects DESC
LIMIT 5
"""

df = pd.read_sql(sql_query, conn)
print(df)


                                          contractor  Total_Projects  On_Time_Projects  Delayed_Projects
0  LEGACY CONSTRUCTION CORPORATION (FORMERLY: LEG...             115                44                71
1               ST. TIMOTHY CONSTRUCTION CORPORATION              83                28                55
2  ALPHA & OMEGA GEN. CONTRACTOR & DEVELOPMENT CORP.              84                33                51
3                                        QM BUILDERS              88                39                49
4       CENTERWAYS CONSTRUCTION AND DEVELOPMENT INC.              64                15                49


Contractor Performance Analysis (Sorted by Cost Control)
- Total Financial Scale: Total_Cost_Formatted - 
    - Financial impact of the contractor's work.
- Original Metrics: On_Time_Projects, Delayed_Projects
    - Relative performance metric, normalizing for the total number of projects.
- Performance Percentage: Delayed_Pct,
- Average Delay Severity (in days): Avg_Delay_Days
    - Severity of the average delay for the contractor.
- Max Delay Severity (in days): Max_Delay_Days
    - Worst-case scenario performance.
- Cost Control Metric: Cost_Over_ABC_Count:
    - How often the awarded cost exceeded the Approved Budget for the Contract.

In [64]:
sql_query = """
WITH ContractorPerformance AS (
    SELECT 
        contractor,
        COUNT(project_id) AS Total_Projects,
        SUM(contract_cost) AS Total_Cost,
        SUM(CASE WHEN completion_date_actual <= completion_date_original THEN 1 ELSE 0 END) AS On_Time_Projects, 
        SUM(CASE WHEN completion_date_actual > completion_date_original THEN 1 ELSE 0 END) AS Delayed_Projects,
        AVG(
            CASE 
                WHEN JULIANDAY(completion_date_actual) > JULIANDAY(completion_date_original) 
                THEN JULIANDAY(completion_date_actual) - JULIANDAY(completion_date_original)
                ELSE NULL -- Exclude on-time/early projects from the average calculation
            END
        ) AS Avg_Days_Late_When_Delayed,
        MAX(JULIANDAY(completion_date_actual) - JULIANDAY(completion_date_original)) AS Max_Delay_Days,
        SUM(CASE WHEN contract_cost > abc THEN 1 ELSE 0 END) AS Cost_Over_ABC_Count
    FROM 
        flood_control_projects 
    WHERE 
        completion_date_actual IS NOT NULL
    GROUP BY 
        contractor
)
-- Main SELECT Statement: Calculates the percentage using the aliases from the CTE
SELECT
    contractor,
    Total_Projects,
    printf('%,.2f', Total_Cost) AS Total_Cost_Formatted, 
    On_Time_Projects, 
    Delayed_Projects,
    -- Now Delayed_Projects is a valid column alias for calculation
    ROUND((CAST(Delayed_Projects AS REAL) * 100.0) / Total_Projects, 2) AS Delayed_Pct,
    ROUND(Avg_Days_Late_When_Delayed, 2) AS Avg_Days_Late_When_Delayed,
    Max_Delay_Days,
    Cost_Over_ABC_Count
FROM 
    ContractorPerformance
ORDER BY 
    Cost_Over_ABC_Count DESC 
LIMIT 5;
"""

df = pd.read_sql(sql_query, conn)
print(df)


                                          contractor  Total_Projects Total_Cost_Formatted  On_Time_Projects  Delayed_Projects  Delayed_Pct  Avg_Days_Late_When_Delayed  Max_Delay_Days  Cost_Over_ABC_Count
0                                     WAWAO BUILDERS              53     3,829,053,280.00                44                 8        15.09                       16.13            82.0                   22
1  SUNWEST, INC. (FORMERLY: SUNWEST CONSTRUCTION ...              67     8,247,444,379.00                50                17        25.37                       12.29           157.0                   20
2               ST. TIMOTHY CONSTRUCTION CORPORATION              83     5,473,908,390.00                28                55        66.27                       16.02           256.0                   18
3  STO. CRISTO CONSTRUCTION & TRADING INC. (FOR: ...              46     3,513,475,069.00                13                33        71.74                        7.00           135.0  

In [71]:
sql_query = """
WITH ContractorPerformance AS (
    SELECT 
        district_engineering_office,
        COUNT(project_id) AS Total_Projects,
        SUM(contract_cost) AS Total_Cost,
        SUM(CASE WHEN completion_date_actual <= completion_date_original THEN 1 ELSE 0 END) AS On_Time_Projects, 
        SUM(CASE WHEN completion_date_actual > completion_date_original THEN 1 ELSE 0 END) AS Delayed_Projects,
        AVG(
            CASE 
                WHEN JULIANDAY(completion_date_actual) > JULIANDAY(completion_date_original) 
                THEN JULIANDAY(completion_date_actual) - JULIANDAY(completion_date_original)
                ELSE NULL -- Exclude on-time/early projects from the average calculation
            END
        ) AS Avg_Days_Late_When_Delayed,
        MAX(JULIANDAY(completion_date_actual) - JULIANDAY(completion_date_original)) AS Max_Delay_Days,
        SUM(CASE WHEN contract_cost > abc THEN 1 ELSE 0 END) AS Cost_Over_ABC_Count
    FROM 
        flood_control_projects 
    WHERE 
        completion_date_actual IS NOT NULL
    GROUP BY 
        district_engineering_office
)
-- Main SELECT Statement: Calculates the percentage using the aliases from the CTE
SELECT
    district_engineering_office,
    Total_Projects,
    printf('%,.2f', Total_Cost) AS Total_Cost_Formatted, 
    On_Time_Projects, 
    Delayed_Projects,
    -- Now Delayed_Projects is a valid column alias for calculation
    ROUND((CAST(Delayed_Projects AS REAL) * 100.0) / Total_Projects, 2) AS Delayed_Pct,
    ROUND(Avg_Days_Late_When_Delayed, 2) AS Avg_Days_Late_When_Delayed,
    Max_Delay_Days,
    Cost_Over_ABC_Count
FROM 
    ContractorPerformance
ORDER BY 
    Cost_Over_ABC_Count DESC 
LIMIT 10;
"""

df = pd.read_sql(sql_query, conn)
print(df)


                   district_engineering_office  Total_Projects Total_Cost_Formatted  On_Time_Projects  Delayed_Projects  Delayed_Pct  Avg_Days_Late_When_Delayed  Max_Delay_Days  Cost_Over_ABC_Count
0      Bulacan 1st District Engineering Office             452    29,183,182,172.00               264               188        41.59                        1.92           165.0                  101
1  Malabon-Navotas District Engineering Office             122     5,155,617,775.00                53                69        56.56                        1.01             2.0                   87
2           Tarlac District Engineering Office             164    11,865,023,716.00                73                90        54.88                        1.87            64.0                   47
3        Albay 1st District Engineering Office             101     5,366,885,766.00                46                55        54.46                        2.27            71.0                   43
4      Bul

In [72]:
sql_query = """
WITH ContractorPerformance AS (
    SELECT 
        district_engineering_office,
        COUNT(project_id) AS Total_Projects,
        SUM(contract_cost) AS Total_Cost,
        SUM(CASE WHEN completion_date_actual <= completion_date_original THEN 1 ELSE 0 END) AS On_Time_Projects, 
        SUM(CASE WHEN completion_date_actual > completion_date_original THEN 1 ELSE 0 END) AS Delayed_Projects,
        AVG(
            CASE 
                WHEN JULIANDAY(completion_date_actual) > JULIANDAY(completion_date_original) 
                THEN JULIANDAY(completion_date_actual) - JULIANDAY(completion_date_original)
                ELSE NULL -- Exclude on-time/early projects from the average calculation
            END
        ) AS Avg_Days_Late_When_Delayed,
        MAX(JULIANDAY(completion_date_actual) - JULIANDAY(completion_date_original)) AS Max_Delay_Days,
        SUM(CASE WHEN contract_cost > abc THEN 1 ELSE 0 END) AS Cost_Over_ABC_Count
    FROM 
        flood_control_projects 
    WHERE 
        completion_date_actual IS NOT NULL
    GROUP BY 
        district_engineering_office
)
-- Main SELECT Statement: Calculates the percentage using the aliases from the CTE
SELECT
    district_engineering_office,
    Total_Projects,
    printf('%,.2f', Total_Cost) AS Total_Cost_Formatted, 
    On_Time_Projects, 
    Delayed_Projects,
    -- Now Delayed_Projects is a valid column alias for calculation
    ROUND((CAST(Delayed_Projects AS REAL) * 100.0) / Total_Projects, 2) AS Delayed_Pct,
    ROUND(Avg_Days_Late_When_Delayed, 2) AS Avg_Days_Late_When_Delayed,
    Max_Delay_Days,
    Cost_Over_ABC_Count
FROM 
    ContractorPerformance
ORDER BY 
    Delayed_Projects DESC 
LIMIT 10;
"""

df = pd.read_sql(sql_query, conn)
print(df)


                    district_engineering_office  Total_Projects Total_Cost_Formatted  On_Time_Projects  Delayed_Projects  Delayed_Pct  Avg_Days_Late_When_Delayed  Max_Delay_Days  Cost_Over_ABC_Count
0       Bulacan 1st District Engineering Office             452    29,183,182,172.00               264               188        41.59                        1.92           165.0                  101
1  Metro Manila 1st District Engineering Office             230    12,352,162,447.00                85               145        63.04                       14.97           255.0                   12
2       Bulacan 2nd District Engineering Office             190    12,399,706,597.00                61               129        67.89                        1.16            13.0                   38
3      Pampanga 3rd District Engineering Office             133     5,462,150,980.00                36                97        72.93                        2.11            91.0                    2
4    

Top 5 Regions by Total Cost

In [67]:
sql_query = """
SELECT region, 
     printf('%,.2f', SUM(contract_cost)) AS Total_Cost 
FROM flood_control_projects
GROUP BY region 
ORDER BY Total_Cost DESC LIMIT 5;
"""

df = pd.read_sql(sql_query, conn)
print(df)


                    region         Total_Cost
0               Region III  98,005,510,571.00
1  National Capital Region  52,663,633,899.00
2                Region IX   5,797,880,431.00
3                 Region V  49,973,685,594.00
4              Region IV-A  40,062,856,382.00


Projects by Province

In [73]:
sql_query = """
SELECT province, COUNT(DISTINCT global_id) AS Project_Count 
FROM flood_control_projects GROUP BY province ORDER BY Project_Count DESC;
"""

df = pd.read_sql(sql_query, conn)
print(df)


            province  Project_Count
0            BULACAN            668
1               CEBU            414
2            ISABELA            341
3         PANGASINAN            313
4           PAMPANGA            292
..               ...            ...
93          GUIMARAS              8
94           QUIRINO              7
95          SIQUIJOR              4
96              SULU              3
97  MANDALUYONG CITY              2

[98 rows x 2 columns]


Cost Distribution by Implementing Office

In [74]:
sql_query = """
SELECT implementing_office, SUM(contract_cost) AS Total_Cost FROM flood_control_projects GROUP BY implementing_office ORDER BY Total_Cost DESC;
"""

df = pd.read_sql(sql_query, conn)
print(df)


                                   implementing_office   Total_Cost
0              Bulacan 1st District Engineering Office  28892332626
1                                          Region IV-B  19041726130
2                                             Region I  13681543675
3             North Manila District Engineering Office  12768889673
4                 Cebu 7th District Engineering Office  12663827775
..                                                 ...          ...
198            Iligan City District Engineering Office    197577184
199  Zamboanga del Norte 4th District Engineering O...    166600000
200             Cavite 2nd District Engineering Office    124557910
201             Ifugao 1st District Engineering Office     37105105
202               Siquijor District Engineering Office     19600000

[203 rows x 2 columns]


Projects by Legislative DistricQt

In [76]:
sql_query = """
SELECT legislative_district, COUNT(DISTINCT global_id) AS Project_Count FROM flood_control_projects GROUP BY legislative_district ORDER BY Project_Count DESC;
"""

df = pd.read_sql(sql_query, conn)
print(df)


                           legislative_district  Project_Count
0         BULACAN (SECOND LEGISLATIVE DISTRICT)            192
1           CEBU (SEVENTH LEGISLATIVE DISTRICT)            171
2          BULACAN (FIRST LEGISLATIVE DISTRICT)            158
3         PAMPANGA (FIRST LEGISLATIVE DISTRICT)            133
4        LA UNION (SECOND LEGISLATIVE DISTRICT)            131
..                                          ...            ...
247             SIQUIJOR (LEGISLATIVE DISTRICT)              4
248         BATAAN (THIRD LEGISLATIVE DISTRICT)              3
249          SULU (SECOND LEGISLATIVE DISTRICT)              2
250  CITY OF MANDALUYONG (LEGISLATIVE DISTRICT)              2
251           SULU (FIRST LEGISLATIVE DISTRICT)              1

[252 rows x 2 columns]


Cost by Infrastructure Type

In [82]:
sql_query = """
SELECT 
    typeof_work, 
    legislative_district,
    SUM(contract_cost) AS Total_Cost, COUNT(DISTINCT global_id) AS Project_Count 
FROM flood_control_projects 
GROUP BY typeof_work, legislative_district
ORDER BY Total_Cost DESC;
"""

df = pd.read_sql(sql_query, conn)
print(df)


                                           typeof_work                            legislative_district  Total_Cost  Project_Count
0           Construction of Flood Mitigation Structure             CEBU (SEVENTH LEGISLATIVE DISTRICT)  8616879641            116
1           Construction of Flood Mitigation Structure          LA UNION (SECOND LEGISLATIVE DISTRICT)  8423532162            119
2           Construction of Flood Mitigation Structure           BULACAN (SECOND LEGISLATIVE DISTRICT)  7913360668            121
3           Construction of Flood Mitigation Structure                     ABRA (LEGISLATIVE DISTRICT)  7603900131            104
4           Construction of Flood Mitigation Structure           ISABELA (FOURTH LEGISLATIVE DISTRICT)  6803054133             85
..                                                 ...                                             ...         ...            ...
902                 Construction of Drainage Structure            CAVITE (SECOND LEGISLATI

Cost vs. ABC (Deviation)

In [84]:
sql_query = """
SELECT 
    legislative_district,
    AVG(ABS(contract_cost - abc)) AS Avg_Cost_ABC_Deviation, 
    SUM(CASE WHEN contract_cost < abc THEN 1 ELSE 0 END) AS Below_ABC_Count 
FROM flood_control_projects 
WHERE abc IS NOT NULL AND contract_cost IS NOT NULL
GROUP BY legislative_district;
"""

df = pd.read_sql(sql_query, conn)
print(df)


                                 legislative_district  Avg_Cost_ABC_Deviation  Below_ABC_Count
0                         ABRA (LEGISLATIVE DISTRICT)            4.747814e+05               66
1       AGUSAN DEL NORTE (FIRST LEGISLATIVE DISTRICT)            2.700950e+04               52
2      AGUSAN DEL NORTE (SECOND LEGISLATIVE DISTRICT)            1.153516e+05               36
3         AGUSAN DEL SUR (FIRST LEGISLATIVE DISTRICT)            2.732023e+04               30
4        AGUSAN DEL SUR (SECOND LEGISLATIVE DISTRICT)            5.986207e+01               18
..                                                ...                     ...              ...
247  ZAMBOANGA DEL NORTE (THIRD LEGISLATIVE DISTRICT)            1.342453e+05                6
248    ZAMBOANGA DEL SUR (FIRST LEGISLATIVE DISTRICT)            2.859633e+05               14
249   ZAMBOANGA DEL SUR (SECOND LEGISLATIVE DISTRICT)            4.093633e+06               10
250    ZAMBOANGA SIBUGAY (FIRST LEGISLATIVE DISTRI